In [48]:
import matplotlib.pyplot as plt
import torch
import torchvision.datasets


In [49]:
def synthetic_data(w,b,num):
    x=torch.normal(0,1,[num,len(w)])
    y = torch.matmul(x, w) + b
    y+=torch.normal(0,0.01,y.shape)
    return x,y.reshape(-1,1)
initial_w=torch.tensor([2,3.4])
initial_b=4.2
features,labels=synthetic_data(initial_w,initial_b,1000)
features,labels

(tensor([[ 1.7092, -1.0945],
         [ 0.9434, -0.8578],
         [ 0.4713,  1.3788],
         ...,
         [ 0.1344, -0.5552],
         [ 0.2254, -2.8115],
         [ 0.5358, -1.1096]]),
 tensor([[ 3.8884e+00],
         [ 3.1800e+00],
         [ 9.8367e+00],
         [ 5.0305e+00],
         [-4.5458e+00],
         [ 5.6213e+00],
         [ 3.1785e+00],
         [ 9.1482e+00],
         [ 5.8156e+00],
         [ 1.0168e+01],
         [-1.3982e+00],
         [ 3.2374e+00],
         [ 4.1450e+00],
         [ 7.1243e+00],
         [ 6.4586e+00],
         [ 6.8196e+00],
         [ 8.1179e+00],
         [ 2.8160e+00],
         [ 4.2018e+00],
         [ 5.3661e+00],
         [ 1.6122e+00],
         [ 7.7683e+00],
         [ 6.2317e+00],
         [ 6.3182e+00],
         [ 1.7505e-01],
         [-7.4541e-01],
         [ 5.3372e+00],
         [-1.1553e-01],
         [ 3.3123e+00],
         [ 7.3339e+00],
         [-1.5817e+00],
         [ 3.0708e+00],
         [ 3.6363e-01],
         [ 9.8231e

In [50]:
len(features[:,1]),len(labels[:,0])

(1000, 1000)

In [51]:
# plt.scatter(features[:,1],labels[:,0])

In [52]:
# import random
# # 本函数已保存在d2lzh包中方便以后使用
# def data_iter(batch_size, features, labels):
#     num_examples = len(features)
#     indices = list(range(num_examples))
#     random.shuffle(indices)  # 样本的读取顺序是随机的
#     for i in range(0, num_examples, batch_size):
#         j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # 最后一次可能不足一个batch
#         yield  features.index_select(0, j), labels.index_select(0, j)
# batch_size = 10
#
# for X, y in data_iter(batch_size, features, labels):
#     print(X, y)
#     break


In [53]:
import random


def data_iter(batch_size,features,labels):
    num=len(features)
    indices=list(range(num))
    random.shuffle(indices)

    for i in range(0,num,batch_size):
        batch_indices=torch.tensor(indices[i:min(i+batch_size,num)])
        return features.index_select(0,batch_indices),labels.index_select(0,batch_indices)
batch_size = 10

x,y=data_iter(batch_size,features,labels)
x,y

(tensor([[-1.5194, -1.2720],
         [ 0.8406,  1.8228],
         [ 1.6554, -0.6453],
         [ 0.5762,  0.3081],
         [-0.7302, -0.5396],
         [ 0.2942, -0.5096],
         [-0.8103,  0.1914],
         [-1.4373, -0.1325],
         [-1.2164, -0.7036],
         [ 0.5469,  1.5810]]),
 tensor([[-3.1690],
         [12.0763],
         [ 5.3094],
         [ 6.3985],
         [ 0.9087],
         [ 3.0510],
         [ 3.2130],
         [ 0.8639],
         [-0.6156],
         [10.6494]]))

In [54]:
w=torch.normal(0,0.01,(2,1),requires_grad=True)
# w=torch.zeros((2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)
w,b

(tensor([[-0.0135],
         [ 0.0120]], requires_grad=True),
 tensor([0.], requires_grad=True))

In [55]:
def linear(x,w,b):
    return torch.matmul(x,w)+b

In [56]:
def squared_loss(y_hat , y):  #@save
    """均方损失"""
    return (y_hat  - y.reshape(y_hat.shape)) ** 2 / 2

In [57]:
lr=0.01
num_epochs=3
net=linear
loss=squared_loss


In [58]:
for epoch in range(num_epochs):
    for i in range(100):
        x,y=data_iter(batch_size,features,labels)
        l=loss(net(x,w,b),y)
        l.sum().backward()
        sdg([w,b],lr,batch_size)
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l=loss(net(features,w,b),labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

NameError: name 'sdg' is not defined

In [ ]:
print(f'w的估计误差: {initial_w - w.reshape(initial_w.shape)}')
print(f'b的估计误差: {initial_b - b}')

In [ ]:
def sdg(params, lr,batch_size):
    for param in params:
        param.data-=(lr*param.grad)/batch_size


简洁实现

In [ ]:
def synthetic_data(w,b,num):
    x=torch.normal(0,1,[num,len(w)])
    y = torch.matmul(x, w) + b
    y+=torch.normal(0,0.01,y.shape)
    return x,y.reshape(-1,1)
initial_w=torch.tensor([2,3.4])
initial_b=4.2
features,labels=synthetic_data(initial_w,initial_b,1000)
features,labels

In [68]:
from torch.utils import data
def load_array(data_arrays,batch_size,is_train=True):
    dataset=data.TensorDataset(*data_arrays)
    print(dataset.tensors)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

data_iter=load_array((features,labels),batch_size)

(tensor([[ 1.7092, -1.0945],
        [ 0.9434, -0.8578],
        [ 0.4713,  1.3788],
        ...,
        [ 0.1344, -0.5552],
        [ 0.2254, -2.8115],
        [ 0.5358, -1.1096]]), tensor([[ 3.8884e+00],
        [ 3.1800e+00],
        [ 9.8367e+00],
        [ 5.0305e+00],
        [-4.5458e+00],
        [ 5.6213e+00],
        [ 3.1785e+00],
        [ 9.1482e+00],
        [ 5.8156e+00],
        [ 1.0168e+01],
        [-1.3982e+00],
        [ 3.2374e+00],
        [ 4.1450e+00],
        [ 7.1243e+00],
        [ 6.4586e+00],
        [ 6.8196e+00],
        [ 8.1179e+00],
        [ 2.8160e+00],
        [ 4.2018e+00],
        [ 5.3661e+00],
        [ 1.6122e+00],
        [ 7.7683e+00],
        [ 6.2317e+00],
        [ 6.3182e+00],
        [ 1.7505e-01],
        [-7.4541e-01],
        [ 5.3372e+00],
        [-1.1553e-01],
        [ 3.3123e+00],
        [ 7.3339e+00],
        [-1.5817e+00],
        [ 3.0708e+00],
        [ 3.6363e-01],
        [ 9.8231e-01],
        [ 6.9599e+00],
        [ 1

In [ ]:
next(iter(data_iter))

In [71]:
from torch import nn
net=nn.Sequential(nn.Linear(2,1))

In [ ]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

In [ ]:
loss=nn.MSELoss()

In [ ]:
trainer=torch.optim.SGD(net.parameters(),lr=0.01)

In [ ]:
for epoch in range(num_epochs):
    for x,y in data_iter:
        l=loss(net(x),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l=loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

In [ ]:
w = net[0].weight.data
print('w的估计误差：', initial_w - w.reshape(initial_w.shape))
b = net[0].bias.data
print('b的估计误差：', initial_b - b)

MINST

In [64]:
import torchvision
import torchvision.transforms as transforms
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())
mnist_test,mnist_train

(Dataset FashionMNIST
     Number of datapoints: 10000
     Root location: C:\Users\sunyu/Datasets/FashionMNIST
     Split: Test
     StandardTransform
 Transform: ToTensor(),
 Dataset FashionMNIST
     Number of datapoints: 60000
     Root location: C:\Users\sunyu/Datasets/FashionMNIST
     Split: Train
     StandardTransform
 Transform: ToTensor())

In [65]:
mnist_train[0][0].size()

torch.Size([1, 28, 28])

In [66]:
feature,label=mnist_train[0]
print(feature.shape,label)

torch.Size([1, 28, 28]) 9


In [ ]:
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [ ]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save
    """绘制图像列表"""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            # 图片张量
            ax.imshow(img.numpy())
        else:
            # PIL图片
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [67]:
X, y = next(iter(data.DataLoader(mnist_train, batch_size=18)))
show_images(X.reshape(18, 28, 28), 2, 9, titles=get_fashion_mnist_labels(y));

NameError: name 'data' is not defined

In [62]:
batch_size=256
threat_num=4
train_iter=data.DataLoader(mnist_train,batch_size,shuffle=True,num_workers=threat_num)
test_iter=data.DataLoader(mnist_test,batch_size,shuffle=True,num_workers=threat_num)

NameError: name 'data' is not defined

In [ ]:
import time

start = time.time()
for X, y in train_iter:
    continue
print('%.2f sec' % (time.time() - start))


In [ ]:
for X, y in train_iter:
    print(X.shape, X.dtype, y.shape, y.dtype)
    break

In [ ]:
num_inputs=784
num_outputs=10

w=torch.normal(0,0.01,[num_inputs,num_outputs],requires_grad=True)
b=torch.zeros(num_outputs,requires_grad=True)
w,b

In [ ]:
def softmax(x):
    x_exp=torch.exp(x)
    partition=x_exp.sum(1,keepdim=True)
    return  x_exp/partition

In [ ]:
x=torch.normal(0,1,[2,5])
x
x_pre=softmax(x)
x_pre,x_pre.sum(1)

In [ ]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), w) + b)


In [ ]:
y=torch.tensor([0,2])
y

In [ ]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

In [ ]:
def accuracy(y_hat,y):
    # return (y_hat.argmax(dim=1)==y).float().mean().item()
    return  (y_hat.argmax(dim=1)==y).float().mean().tolist()

In [ ]:
print(accuracy(y_hat, y))

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n


In [ ]:
print(evaluate_accuracy(test_iter, net))


In [ ]:
num_epochs, lr = 5, 0.1

# 本函数已保存在d2lzh包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()
            if optimizer is None:
                sdg(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [w, b], lr)


In [ ]:
def predict_ch3(net, test_iter, n=6):  #@save
    """预测标签（定义见第3章）"""
    for X, y in test_iter:
        break
    trues = get_fashion_mnist_labels(y)
    preds = get_fashion_mnist_labels(net(X).argmax(axis=1))
    titles = [true +'\n' + pred for true, pred in zip(trues, preds)]
    show_images(
        X[0:n].reshape((n, 28, 28)), 1, n, titles=titles[0:n])

predict_ch3(net, test_iter,10)

In [82]:
import torchvision
import torchvision.transforms as transforms
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())
batch_size=256
threat_num=4
train_iter=data.DataLoader(mnist_train,batch_size,shuffle=True,num_workers=threat_num)
test_iter=data.DataLoader(mnist_test,batch_size,shuffle=True,num_workers=threat_num)

In [77]:
num_inputs = 784
num_outputs = 10

net=nn.Sequential(nn.Flatten(),nn.Linear(num_inputs,num_outputs))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
net.apply(init_weights)




Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
)

In [73]:
loss=nn.CrossEntropyLoss()

In [74]:
trainer=torch.optim.SGD(net.parameters(),0.01)

In [83]:
def train_softmax(net,train_iter,test_iter,loss,num_epochs,optimizer):
    l1=[]
    l2=[]
    for epoch in range(num_epochs):
        sum_loss,acc_train,n_train,acc_test,n_test=0,0,0,0,0
        for x,y in train_iter:
            k=net(x)
            l=loss(k,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            sum_loss+=l.item()
            acc_train+=(k.argmax(dim=1)==y).float().sum().item()
            n_train+=y.shape[0]
        for x,y in test_iter:
            k=net(x)
            acc_test+=(k.argmax(dim=1)==y).float().sum().item()
            n_test+=y.shape[0]
        # print("epoch: "+str(epoch+1))
        # print("acc_train: "+ str(acc_train/n_train))
        # print("test_acc: "+str(acc_test/n_test))
        l1.append(acc_train/n_train)
        l2.append(acc_test/n_test)
    plt.plot(range(10),l1)
    plt.plot(range(10),l2)
    plt.legend(['train_acc','test_acc'])
    plt.show()

In [84]:
num_epochs = 10
train_softmax(net,train_iter,test_iter,loss,num_epochs,trainer)

tensor([2, 2, 2, 1, 1, 2, 1, 3, 1, 2, 3, 3, 1, 1, 1, 3, 2, 1, 1, 3, 1, 2, 2, 1,
        3, 2, 1, 1, 3, 1, 2, 2, 2, 3, 1, 2, 1, 1, 1, 1, 1, 3, 1, 2, 1, 2, 3, 1,
        3, 2, 2, 1, 3, 3, 3, 3, 1, 1, 1, 2, 2, 1, 1, 1, 3, 2, 3, 3, 1, 3, 1, 3,
        1, 1, 3, 3, 3, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 1, 1, 3, 3, 3, 1, 3, 2, 1,
        1, 2, 3, 1, 1, 2, 1, 1, 1, 1, 3, 1, 2, 2, 2, 3, 1, 1, 3, 1, 3, 1, 3, 1,
        2, 1, 2, 2, 2, 3, 1, 3, 3, 3, 1, 3, 1, 1, 2, 2, 2, 1, 1, 1, 3, 2, 2, 3,
        5, 3, 1, 3, 2, 1, 2, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1,
        2, 1, 2, 1, 2, 2, 1, 2, 1, 3, 3, 3, 3, 1, 1, 1, 1, 2, 3, 3, 1, 3, 1, 1,
        1, 1, 3, 2, 2, 1, 3, 1, 1, 2, 1, 2, 3, 3, 1, 2, 1, 3, 2, 1, 2, 1, 0, 3,
        2, 3, 1, 3, 3, 1, 2, 3, 2, 2, 2, 3, 3, 2, 1, 9, 3, 3, 2, 2, 2, 1, 9, 3,
        1, 1, 1, 2, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 2])
tensor([2, 3, 3, 3, 2, 1, 2, 2, 3, 3, 3, 1, 1, 2, 1, 3, 3, 2, 3, 1, 1, 1, 3, 1,
        1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 3, 2, 1, 1, 2, 3

KeyboardInterrupt: 